# 2 types of PDFs encountered so far
## type1: paragraph breaks are well formed... can potentially leverage logic to find headers to logically chunk the text up
## type 2: line breaks all over the place

In [1]:
import PyPDF2
import re
import pandas as pd

In [2]:

folder = 'C:/Users/ericf/OneDrive/Documents/analytic-projects/red cross/rc_downloaded_materials/evaluations/'
file = 'IFRC Ukraine crisis response_CVA review report_ final.pdf'
file = 'Afghanistan_Humanitarian_Crises_MTR report_final.pdf' #very few newlines
#file = 'Lessons Learned CVTL COVID-19 Ops-1.pdf'

filepath = f'{folder}{file}'



In [3]:
def read_document(filepath, begin_page=0, end_page=-1) -> str:
    pdfFileObj = open(filepath, 'rb')
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    end_page = end_page if end_page > 0 else pdfReader.numPages
    
    
    page = ''
    for i in range (begin_page, end_page):
        pageObj = pdfReader.getPage(i)
        page += ' ' + pageObj.extractText()
        
    return page


document = read_document(filepath)

In [5]:

def split_to_paragraphs(text, limit=100):
    
    
    #clean up errant situations where two strings are merged without a space
    text = re.sub(r'([a-z])([A-Z])', r'\1. \2', text)
    
    
    # determine if newlines are used solely in the separation of paragraphs
    # or as is common with pdf, also used to format line-lengths

    chars_per_newline = len(text) / len(text.split('\n')) 
    if chars_per_newline < limit:
        print('excess_newlines')
        #split the doc on sentence terminators followed by newline
        paragraphs = re.split('[.?!]\s*\n', text)
    else:
        print('normal_newlines')
        paragraphs = re.split("\s{2,}", text)
        
    #strip out any remaining newlines
    for i in range(len(paragraphs)):
        paragraphs[i] = paragraphs[i].replace('\n',' ')

    #create a dataframe with each paragraph as its own record
    df_paragraphs = pd.DataFrame([{"file" : file, "paragraph" : paragraph}
                             for paragraph in paragraphs if paragraph])
    
    return df_paragraphs

df_paragraphs = split_to_paragraphs(document)
print(len(df_paragraphs))

normal_newlines
397


In [ ]:
df_paragraphs

In [ ]:
#Still working on this
def identify_ordered_lists(text):
      #regular expression to find numbered lists with the format "1. text"
    list_pat = re.compile('\.(\s+[\d]+\. )([^.!?]*)')
    list_pat.findall(mixed)